# Comparação de Algoritmos de Busca em Grafos — IA UFOP

**Problema:** Roteamento em Ouro Preto com função de custo (declividade, rugosidade, congestionamento).

**Algoritmos:** Dijkstra, A* e D* Lite.

**Cenários:** baseline, evento (ruas interditadas), clima (chuva — ladeiras com peso 200%).

In [56]:
import sys, os
sys.path.insert(0, '.')
from src.graph import build_ouro_preto_example
from src.algorithms import dijkstra, a_star, d_star_lite
from src.scenarios import (
    apply_event_scenario, apply_climate_scenario, apply_climate_scenario_by_region,
    apply_congestion_scenario_by_edge, apply_barriers, reset_scenarios,
    OURO_PRETO_BLOCKED_CASE1, OURO_PRETO_CONGESTED_CASE2, OURO_PRETO_RAIN_REGIONS_CASE3,
    OURO_PRETO_BLOCKED_CASE4, OURO_PRETO_CONGESTED_CASE4, OURO_PRETO_BLOCKED_CASE5,
    OURO_PRETO_CONGESTED_CASE6, OURO_PRETO_BLOCKED_CASE7, OURO_PRETO_RAIN_REGIONS_CASE7,
)
from src.metrics import measure_latency_ms

## 1. Grafo de exemplo (Ouro Preto)

Vértices: Praça Tiradentes, Terminal, Campus UFOP, Rua São José, Diogo de Vasconcelos, Rua do Pilar, Xavier da Veiga, Ladeira da Barra.
Arestas: distância, declividade (%), rugosidade (pé de moleque ~1.57), volume/capacidade.

In [57]:
api_key = os.getenv('GOOGLE_MAPS_API_KEY')
G = build_ouro_preto_example(api_key)
print('Vértices:', list(G.nodes()))
print('Arestas (origem -> destino):', list(G.edges()))

Vértices: ['sao_jose', 'conde_bobadela', 'rua_pilar', 'xavier_veiga', 'padre_rolim', 'claudio_manoel', 'conselheiro_quintiliano', 'paulistas', 'getulio_vargas', 'henri_gorceix', 'joao_de_paiva', 'teixeira_amaral', 'coronel_alves', 'conego_trindade', 'direita', 'bernardo_vasconcelos', 'sao_francisco_assis', 'dom_silverio', 'camilo_brito', 'parana', 'alvarenga', 'flores', 'conceicao', 'ouro', 'cruz', 'rosario', 'merces', 'chafariz', 'sol', 'prata', 'sao_miguel', 'santa_rita', 'lajes', 'ladeira_barra', 'ladeira_praca', 'ladeira_ameno_resgalla', 'ladeira_santa_efigenia', 'ladeira_custodio', 'ladeira_piedade', 'morro_da_queimada', 'rio_piracicaba', 'rio_itabira', 'treze_maio', 'vinte_e_quatro_de_junho', 'quinze_de_agosto', 'silviano_brandao', 'santos_dumont', 'pedro_i', 'americo_rene_gianetti', 'simao_lacerda', 'hugo_soderi', 'domingos_mendes', 'mingon_campos', 'cristo_rei', 'alberto_ansaloni', 'rua_quatorze', 'rua_onze', 'rua_dois', 'rua_tres', 'rua_quatro', 'rua_nove', 'joao_fernandes_vie

In [58]:
from src.helpers.graph_helper import display_graph

START, GOAL = 'americo_rene_gianetti', 'rio_piracicaba'
# Gera grafo interativo (Dash Cytoscape) e exibe no notebook
display_graph(G, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500, start=START, goal=GOAL)

## 2. Cenário baseline — comparação dos três algoritmos

In [59]:
reset_scenarios(G)

lat_d, (path_d, cost_d) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a, (path_a, cost_a) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl, (path_dl, cost_dl) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('Dijkstra:  caminho =', path_d, '| custo =', round(cost_d, 2))
print('A*:       caminho =', path_a, '| custo =', round(cost_a, 2))
print('D* Lite:  caminho =', path_dl, '| custo =', round(cost_dl, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d:.4f} ms | A*: {lat_a:.4f} ms | D* Lite: {lat_dl:.4f} ms')

Dijkstra:  caminho = ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7059.91
A*:       caminho = ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7059.91
D* Lite:  caminho = ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7059.91
  Latência média (200 exec.): Dijkstra: 12.4823 ms | A*: 5.8805 ms | D* Lite: 17.9442 ms


## 3. Casos de teste com impeditivos no caminho (Ouro Preto)

Mantendo **START=rio_piracicaba** e **GOAL=americo_rene_gianetti**. Cada caso aplica interdições, congestionamento ou chuva em ruas do caminho baseline (rio_piracicaba → joao_de_paiva → hugo_soderi → americo_rene_gianetti) para forçar desvio ou custo maior.

In [60]:
def run_case(name, setup_fn):
    """Aplica cenário, roda os 3 algoritmos e retorna (path_d, cost_d, path_a, cost_a, path_dl, cost_dl)."""
    reset_scenarios(G)
    setup_fn(G)
    l_d, (path_d, cost_d) = measure_latency_ms(lambda: dijkstra(G, START, GOAL))
    l_a, (path_a, cost_a) = measure_latency_ms(lambda: a_star(G, START, GOAL))
    l_dl, (path_dl, cost_dl) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL))
    print(f"  {name}")
    print(f"    Dijkstra:  {path_d} | latencia = {l_d:.4f} ms | custo = {round(cost_d, 2)}")
    print(f"    A*:        {path_a} | latencia = {l_a:.4f} ms | custo = {round(cost_a, 2)}")
    print(f"    D* Lite:   {path_dl} | latencia = {l_dl:.4f} ms | custo = {round(cost_dl, 2)}")
    return path_d, cost_d, path_a, cost_a, path_dl, cost_dl

In [61]:
# Caso 1: interdição rio_piracicaba ↔ joao_de_paiva
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case(
    "Caso 1 — interdição (rio_piracicaba ↔ joao_de_paiva)",
    lambda g: apply_barriers(g, OURO_PRETO_BLOCKED_CASE1),
)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso1.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

  Caso 1 — interdição (rio_piracicaba ↔ joao_de_paiva)
    Dijkstra:  ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 13.1432 ms | custo = 7059.91
    A*:        ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 5.9493 ms | custo = 7059.91
    D* Lite:   ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 18.2855 ms | custo = 7059.91


In [62]:
# Caso 2: congestionamento em trechos do caminho
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case(
    "Caso 2 — congestionamento (joao_de_paiva↔hugo_soderi, hugo_soderi↔americo_rene_gianetti)",
    lambda g: apply_congestion_scenario_by_edge(g, OURO_PRETO_CONGESTED_CASE2),
)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso2.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

  Caso 2 — congestionamento (joao_de_paiva↔hugo_soderi, hugo_soderi↔americo_rene_gianetti)
    Dijkstra:  ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 13.7032 ms | custo = 7059.91
    A*:        ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 6.6500 ms | custo = 7059.91
    D* Lite:   ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 19.2947 ms | custo = 7059.91


In [63]:
# Caso 3: chuva em regiões (joao_de_paiva, hugo_soderi, americo_rene_gianetti)
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case(
    "Caso 3 — chuva em regiões do caminho",
    lambda g: apply_climate_scenario_by_region(g, OURO_PRETO_RAIN_REGIONS_CASE3),
)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso3.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

  Caso 3 — chuva em regiões do caminho
    Dijkstra:  ['americo_rene_gianetti', 'hugo_soderi', 'bernardo_vasconcelos', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 10.7388 ms | custo = 7261.03
    A*:        ['americo_rene_gianetti', 'hugo_soderi', 'bernardo_vasconcelos', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 5.9202 ms | custo = 7261.03
    D* Lite:   ['americo_rene_gianetti', 'hugo_soderi', 'bernardo_vasconcelos', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 19.4638 ms | custo = 7261.03


In [64]:
# Caso 4: interdição + congestionamento
def setup_case4(g):
    apply_barriers(g, OURO_PRETO_BLOCKED_CASE4)
    apply_congestion_scenario_by_edge(g, OURO_PRETO_CONGESTED_CASE4)
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case("Caso 4 — interdição + congestionamento", setup_case4)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso4.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

  Caso 4 — interdição + congestionamento
    Dijkstra:  ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 16.1565 ms | custo = 7059.91
    A*:        ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 7.4820 ms | custo = 7059.91
    D* Lite:   ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 20.5677 ms | custo = 7059.91


In [65]:
# Caso 5: interdição joao_de_paiva ↔ hugo_soderi
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case(
    "Caso 5 — interdição (joao_de_paiva ↔ hugo_soderi)",
    lambda g: apply_barriers(g, OURO_PRETO_BLOCKED_CASE5),
)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso5.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

  Caso 5 — interdição (joao_de_paiva ↔ hugo_soderi)
    Dijkstra:  ['americo_rene_gianetti', 'hugo_soderi', 'bernardo_vasconcelos', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 18.2405 ms | custo = 7214.14
    A*:        ['americo_rene_gianetti', 'hugo_soderi', 'bernardo_vasconcelos', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 8.2276 ms | custo = 7214.14
    D* Lite:   ['americo_rene_gianetti', 'hugo_soderi', 'bernardo_vasconcelos', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 23.6958 ms | custo = 7214.14


In [66]:
# Caso 6: congestionamento em todas as arestas do caminho baseline
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case(
    "Caso 6 — congestionamento em todo o caminho baseline",
    lambda g: apply_congestion_scenario_by_edge(g, OURO_PRETO_CONGESTED_CASE6),
)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso6.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

  Caso 6 — congestionamento em todo o caminho baseline
    Dijkstra:  ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 17.0014 ms | custo = 7059.91
    A*:        ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 7.7157 ms | custo = 7059.91
    D* Lite:   ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 21.3527 ms | custo = 7059.91


In [67]:
# Caso 7: interdição hugo_soderi↔americo_rene_gianetti + chuva em regiões
def setup_case7(g):
    apply_barriers(g, OURO_PRETO_BLOCKED_CASE7)
    apply_climate_scenario_by_region(g, OURO_PRETO_RAIN_REGIONS_CASE7)
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case("Caso 7 — interdição + chuva em regiões", setup_case7)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso7.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

  Caso 7 — interdição + chuva em regiões
    Dijkstra:  ['americo_rene_gianetti', 'domingos_mendes', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 18.0029 ms | custo = 7316.25
    A*:        ['americo_rene_gianetti', 'domingos_mendes', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 7.8946 ms | custo = 7316.25
    D* Lite:   ['americo_rene_gianetti', 'domingos_mendes', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | latencia = 23.4015 ms | custo = 7316.25


## 3. Cenário de evento — ruas interditadas (Diogo de Vasconcelos)

O sistema deve sugerir rotas alternativas (Rua do Pilar ou Xavier da Veiga).

In [68]:
apply_event_scenario(G)

lat_d_e, (path_d_e, cost_d_e) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_e, (path_a_e, cost_a_e) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_e, (path_dl_e, cost_dl_e) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('Com ruas interditadas:')
print('Dijkstra:  caminho =', path_d_e, '| custo =', round(cost_d_e, 2))
print('A*:       caminho =', path_a_e, '| custo =', round(cost_a_e, 2))
print('D* Lite:  caminho =', path_dl_e, '| custo =', round(cost_dl_e, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_e:.4f} ms | A*: {lat_a_e:.4f} ms | D* Lite: {lat_dl_e:.4f} ms')

Com ruas interditadas:
Dijkstra:  caminho = ['americo_rene_gianetti', 'domingos_mendes', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7316.25
A*:       caminho = ['americo_rene_gianetti', 'domingos_mendes', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7316.25
D* Lite:  caminho = ['americo_rene_gianetti', 'domingos_mendes', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7316.25
  Latência média (200 exec.): Dijkstra: 10.1104 ms | A*: 5.5216 ms | D* Lite: 20.4397 ms


## 4. Cenário climático — chuva (ladeiras íngremes +200%)

In [69]:
reset_scenarios(G)
apply_climate_scenario(G, rain_multiplier=2.0)

lat_d_c, (path_d_c, cost_d_c) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_c, (path_a_c, cost_a_c) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_c, (path_dl_c, cost_dl_c) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('Com chuva (ladeiras +200%):')
print('Dijkstra:  caminho =', path_d_c, '| custo =', round(cost_d_c, 2))
print('A*:       caminho =', path_a_c, '| custo =', round(cost_a_c, 2))
print('D* Lite:  caminho =', path_dl_c, '| custo =', round(cost_dl_c, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_c:.4f} ms | A*: {lat_a_c:.4f} ms | D* Lite: {lat_dl_c:.4f} ms')

Com chuva (ladeiras +200%):
Dijkstra:  caminho = ['americo_rene_gianetti', 'hugo_soderi', 'bernardo_vasconcelos', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7325.59
A*:       caminho = ['americo_rene_gianetti', 'hugo_soderi', 'bernardo_vasconcelos', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7325.59
D* Lite:  caminho = ['americo_rene_gianetti', 'hugo_soderi', 'bernardo_vasconcelos', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7325.59
  Latência média (200 exec.): Dijkstra: 11.8966 ms | A*: 5.6418 ms | D* Lite: 23.6727 ms


### 4.1 Cenários combinados (Ouro Preto)

Bloqueio + chuva; bloqueio + chuva + alta densidade; chuva e congestionamento **por região** (alguns bairros com chuva/trânsito, outros não).

In [70]:
# Cenário A: bloqueio (Diogo de Vasconcelos) + chuva
reset_scenarios(G)
apply_event_scenario(G)
apply_climate_scenario(G, rain_multiplier=2.0)

lat_d_ac, (path_d_ac, cost_d_ac) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_ac, (path_a_ac, cost_a_ac) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_ac, (path_dl_ac, cost_dl_ac) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)
display_graph(G, start=START, goal=GOAL, path=path_d_ac, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

print('A — Bloqueio + Chuva:')
print('  Dijkstra:', path_d_ac, '| custo =', round(cost_d_ac, 2))
print('  A*:     ', path_a_ac, '| custo =', round(cost_a_ac, 2))
print('  D* Lite:', path_dl_ac, '| custo =', round(cost_dl_ac, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_ac:.4f} ms | A*: {lat_a_ac:.4f} ms | D* Lite: {lat_dl_ac:.4f} ms')

A — Bloqueio + Chuva:
  Dijkstra: ['americo_rene_gianetti', 'hugo_soderi', 'bernardo_vasconcelos', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7325.59
  A*:      ['americo_rene_gianetti', 'hugo_soderi', 'bernardo_vasconcelos', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7325.59
  D* Lite: ['americo_rene_gianetti', 'hugo_soderi', 'bernardo_vasconcelos', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7325.59
  Latência média (200 exec.): Dijkstra: 11.1503 ms | A*: 6.5099 ms | D* Lite: 23.2121 ms


In [71]:
reset_scenarios(G)
apply_event_scenario(G)
apply_climate_scenario(G, rain_multiplier=2.0)
apply_congestion_scenario_by_edge(G, {("lagoa", "morro_santana"): 2.0})

lat_d_b, (path_d_b, cost_d_b) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_b, (path_a_b, cost_a_b) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_b, (path_dl_b, cost_dl_b) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)
display_graph(G, path=path_d_b, start=START, goal=GOAL, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)


print('B — Bloqueio + Chuva + Congestionamento:')
print('  Dijkstra:', path_d_b, '| custo =', round(cost_d_b, 2))
print('  A*:     ', path_a_b, '| custo =', round(cost_a_b, 2))
print('  D* Lite:', path_dl_b, '| custo =', round(cost_dl_b, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_b:.4f} ms | A*: {lat_a_b:.4f} ms | D* Lite: {lat_dl_b:.4f} ms')

B — Bloqueio + Chuva + Congestionamento:
  Dijkstra: ['americo_rene_gianetti', 'hugo_soderi', 'bernardo_vasconcelos', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7325.59
  A*:      ['americo_rene_gianetti', 'hugo_soderi', 'bernardo_vasconcelos', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7325.59
  D* Lite: ['americo_rene_gianetti', 'hugo_soderi', 'bernardo_vasconcelos', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7325.59
  Latência média (200 exec.): Dijkstra: 11.0413 ms | A*: 7.3185 ms | D* Lite: 23.2493 ms


In [72]:
# Cenário C: chuva por região + congestionamento por via (vias centro↔são_jose congestionadas)
reset_scenarios(G)
apply_climate_scenario_by_region(G, {"centro":9.0, "praca_tiradentes": 9.0, "campus": 1.0})
apply_congestion_scenario_by_edge(G, {("centro", "sao_jose"): 5.0, ("sao_jose", "centro"): 5.0})
display_graph(G, start=START, goal=GOAL, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

lat_d_c2, (path_d_c2, cost_d_c2) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_c2, (path_a_c2, cost_a_c2) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_c2, (path_dl_c2, cost_dl_c2) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('C — Chuva por região + Congestionamento por via (centro↔são_jose congestionados):')
print('  Dijkstra:', path_d_c2, '| custo =', round(cost_d_c2, 2))
print('  A*:     ', path_a_c2, '| custo =', round(cost_a_c2, 2))
print('  D* Lite:', path_dl_c2, '| custo =', round(cost_dl_c2, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_c2:.4f} ms | A*: {lat_a_c2:.4f} ms | D* Lite: {lat_dl_c2:.4f} ms')

C — Chuva por região + Congestionamento por via (centro↔são_jose congestionados):
  Dijkstra: ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7059.91
  A*:      ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7059.91
  D* Lite: ['americo_rene_gianetti', 'hugo_soderi', 'joao_de_paiva', 'rio_itabira', 'rio_piracicaba'] | custo = 7059.91
  Latência média (200 exec.): Dijkstra: 11.1724 ms | A*: 6.2465 ms | D* Lite: 18.4709 ms


## 5. Latência de re-roteamento (< 100 ms para tempo real)

In [73]:
reset_scenarios(G)
repetitions = 500

lat_d, (path_d, cost_d) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), repetitions)
lat_a, (path_a, cost_a) = measure_latency_ms(lambda: a_star(G, START, GOAL), repetitions)
lat_dl, (path_dl, cost_dl) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), repetitions)

print(f'Latência média ({repetitions} execuções):')
print(f'  Dijkstra: {lat_d:.4f} ms')
print(f'  A*:       {lat_a:.4f} ms')
print(f'  D* Lite:  {lat_dl:.4f} ms')
print('  Meta: < 100 ms para uso em tempo real')

Latência média (500 execuções):
  Dijkstra: 10.6296 ms
  A*:       5.7839 ms
  D* Lite:  18.3774 ms
  Meta: < 100 ms para uso em tempo real


## 6. Tabela resumo para o relatório

In [74]:
import pandas as pd

reset_scenarios(G)
rows = []
for name, fn in [('Dijkstra', dijkstra), ('A*', a_star), ('D* Lite', d_star_lite)]:
    lat, (path, cost) = measure_latency_ms(lambda: fn(G, START, GOAL), 200)
    rows.append({'Algoritmo': name, 'Custo': round(cost, 2), 'Latência (ms)': round(lat, 4)})

df = pd.DataFrame(rows)
df

,Algoritmo,Custo,Latência (ms)
0,Dijkstra,7059.91,10.2317
1,A*,7059.91,5.5251
2,D* Lite,7059.91,17.2445
